In [1]:
!pip install torch==1.7.1
# !pip install -q --user torch==1.4.0 -f
!pip install transformers
!pip install sentencepiece==0.1.94
!pip install ipywidgets
!pip install rouge
!pip install python-meteor

In [4]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = model.to('cuda' if torch.cuda.is_available() else "cpu")

In [6]:
#Load dataset
WikiHow_sample_leq512 = pd.read_csv('WikiHow_sample_leq512_withsummary.csv')
WikiHow_sample_in1024 = pd.read_csv('WikiHow_sample_in1024_withsummary.csv')
WikiHow_sample_in2048 = pd.read_csv('WikiHow_sample_in2048_withsummary.csv')

In [8]:
#Prepare the model
def preprocess(text):
    preprocess_text = text.strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    return t5_prepared_Text, len(preprocess_text.split())

def summarize(t5_prepared_Text,ml):
    tokenized_text = tokenizer.encode(t5_prepared_Text,max_length=2048,truncation=True, return_tensors="pt").to(device)

    summary_ids = model.generate(tokenized_text,
              max_length= ml,
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

In [10]:
#This is used to split the dataset into multiple epochs to prevent server crash
def get_epoch(which_epoch, epoch_amount, data_length):
    epoch_size = math.floor(data_length/epoch_amount)
    if which_epoch == epoch_amount:
        indecies = list(range((which_epoch-1)*epoch_size))
    else:
        indecies = list(range((which_epoch-1)*epoch_size, which_epoch*epoch_size))
    return indecies

In [ ]:
def get_summary(data,which_epoch, epoch_amount):
    data_length = len(data)
    epoch = get_epoch(which_epoch, epoch_amount, data_length)
    for i,text in zip(epoch,data['text'][epoch]):
        t5_prepared_Text,train_length = preprocess(text)
        ml = round(0.33 * train_length)
        data.loc[i,'train_length'] = train_length
        data.loc[i,'summary'] = summarize(t5_prepared_Text,ml)
    return data

In [53]:
epoch = get_epoch(1,8,len(WikiHow_sample_in2048))
for i,text in zip(epoch,WikiHow_sample_in2048['text'][epoch]):
    t5_prepared_Text,train_length = preprocess(text)
    ml = round(0.33 * train_length)
    WikiHow_sample_in2048['train_length'][i] = train_length
    WikiHow_sample_in2048['summary'][i] = summarize(t5_prepared_Text,ml)

/var/folders/yf/zj5bn4xd37q4x_zscg34zr800000gn/T/ipykernel_3377/384447071.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WikiHow_sample_in2048['train_length'][i] = train_length
/var/folders/yf/zj5bn4xd37q4x_zscg34zr800000gn/T/ipykernel_3377/384447071.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WikiHow_sample_in2048['summary'][i] = summarize(t5_prepared_Text,ml)


In [79]:
#save results
WikiHow_sample_leq512.to_csv('WikiHow_sample_leq512_withsummary.csv')
WikiHow_sample_in1024.to_csv('WikiHow_sample_in1024_withsummary.csv')
WikiHow_sample_in2048.to_csv('WikiHow_sample_in2048_withsummary.csv')